<a href="https://colab.research.google.com/github/ShefaliDhingra/AIRRA/blob/main/Data_Processing_AIRRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q groq ipywidgets
import ipywidgets as widgets
from IPython.display import display, Markdown
from groq import Groq
import os


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 64.1 MB/s eta 0:00:00


In [ ]:
os.environ["GROQ_API_KEY"] = "Hidden for Public Access"
client = Groq(api_key=os.environ["GROQ_API_KEY"])

In [ ]:
# Required Libraries
import io
import requests
import pandas as pd

url = 'https://drive.google.com/file/d/1bKCswm1QiJX9h15AP3i33Qdhirw5d_qA/view?usp=sharing'

file_id = url.split('/')[-2]

download_url = f"https://drive.google.com/uc?id={file_id}"

# Download the File Content
response = requests.get(download_url)
response.raise_for_status()  # Raise an Exception for Bad Responses

# Read the Test Dataset
df = pd.read_csv(io.StringIO(response.text), index_col=False)

In [ ]:
df.head()

,Job Applicant Name,Age,Gender,Race,Ethnicity,Resume,Job Roles,Job Description,Best Match
0,Daisuke Mori,29,Male,Mongoloid/Asian,Vietnamese,"Proficient in Injury Prevention, Motivation, N...",Fitness Coach,A Fitness Coach is responsible for helping cl...,0
1,Taichi Shimizu,31,Male,Mongoloid/Asian,Filipino,"Proficient in Healthcare, Pharmacology, Medica...",Physician,"Diagnose and treat illnesses, prescribe medica...",0
2,Sarah Martin,46,Female,White/Caucasian,Dutch,"Proficient in Forecasting, Financial Modelling...",Financial Analyst,"As a Financial Analyst, you will be responsibl...",0
3,Keith Hughes,43,Male,Negroid/Black,Caribbean,"Proficient in Budgeting, Supply Chain Optimiza...",Supply Chain Manager,A Supply Chain Manager oversees the entire sup...,1
4,James Davis,49,Male,White/Caucasian,English,"Proficient in Logistics, Negotiation, Procurem...",Supply Chain Manager,A Supply Chain Manager oversees the entire sup...,1


In [ ]:
# Split the DataFrame based on the 'Best Match' column
df_accepted = df[df['Best Match'] == 1].copy()

print("\nDataFrame where 'Best Match' is 1:")
display(df_accepted.head())


DataFrame where 'Best Match' is 1:


,Job Applicant Name,Age,Gender,Race,Ethnicity,Resume,Job Roles,Job Description,Best Match
3,Keith Hughes,43,Male,Negroid/Black,Caribbean,"Proficient in Budgeting, Supply Chain Optimiza...",Supply Chain Manager,A Supply Chain Manager oversees the entire sup...,1
4,James Davis,49,Male,White/Caucasian,English,"Proficient in Logistics, Negotiation, Procurem...",Supply Chain Manager,A Supply Chain Manager oversees the entire sup...,1
6,Aaliyah Scott,34,Female,Negroid/Black,Caribbean,"Proficient in 3D Modeling, Project Management,...",Architect,An Architect designs and oversees the construc...,1
10,Brian King,30,Male,White/Caucasian,French,"Proficient in Penetration Testing, Incident Re...",Cybersecurity Analyst,"As a Cybersecurity Analyst, you will safeguard...",1
11,Jonah Anderson,39,Male,White/Caucasian,French,"Proficient in Software Development, Java, Pyth...",Software Engineer,"As a Software Engineer, you will leverage your...",1


In [ ]:
columns_to_drop = ['Age','Best Match', 'Job Applicant Name', 'Race', 'Ethnicity']
df_accepted = df_accepted.drop(columns=columns_to_drop)

In [ ]:
print("\ndf_accepted after dropping columns:")
display(df_accepted.head())


df_accepted after dropping columns:


,Gender,Resume,Job Roles,Job Description
3,Male,"Proficient in Budgeting, Supply Chain Optimiza...",Supply Chain Manager,A Supply Chain Manager oversees the entire sup...
4,Male,"Proficient in Logistics, Negotiation, Procurem...",Supply Chain Manager,A Supply Chain Manager oversees the entire sup...
6,Female,"Proficient in 3D Modeling, Project Management,...",Architect,An Architect designs and oversees the construc...
10,Male,"Proficient in Penetration Testing, Incident Re...",Cybersecurity Analyst,"As a Cybersecurity Analyst, you will safeguard..."
11,Male,"Proficient in Software Development, Java, Pyth...",Software Engineer,"As a Software Engineer, you will leverage your..."


In [ ]:
system_prompt = """
You are a highly skilled HR Assistant and Resume Analyst with extensive experience in evaluating resumes against job descriptions.
Your goal is to extract structured information in the format below.
In the data provided to you (df_accepted), each applicant was **accepted** for the **Job Role** they applied for.

Use both the **Resume** and the corresponding **Job Role** and **Job Description** fields to guide your extraction.
Focus on how the applicant’s qualifications align with the role and description.

Output must be in **valid JSON format**:

{
  "skills": [list of relevant soft or technical skills],
  "technologies": [list of tools, programming languages, or platforms],
  "certifications": [list of certifications or licenses],
  "notable_projects": [list of short project titles or summaries],
  "years_experience": "approximate total years of professional experience",
  "education_level": "highest degree or qualification mentioned"
}

Instructions:
- Use lowercase for JSON keys.
- Use double quotes `"` for all strings and keys.
- If a field is not present, output an empty list `[]` or empty string `""`.
- Keep reasoning concise, objective, and based only on skills, experience, certifications, and alignment with the role.
- Do not add extra commentary outside JSON. Only return JSON.
End of instructions.
"""



In [60]:
# System Prompt commented

# import time, json
# parsed_outputs = []

# for i, row in df_accepted.head(1).iterrows():
#     try:
#         # Pass Resume + Job Role + Job Description
#         user_input = f"""
#         Resume: {row['Resume']}
#         Job Role: {row['Job Roles']}
#         Job Description: {row['Job Description']}
#         """

#         response = client.chat.completions.create(
#             model="llama-3.3-70b-versatile",
#             messages=[
#                 {"role": "system", "content": system_prompt},
#                 {"role": "user", "content": user_input},
#             ],
#             temperature=0.2
#         )

#         # Try parsing response to JSON
#         response_text = response.choices[0].message.content.strip()
#         print(f"Response for index {i}: {response_text}")
#         response_json = json.loads(response_text)
#         parsed_outputs.append(response_json)

#     except Exception as e:
#         print(f"Error processing index {i}: {e}")
#         parsed_outputs.append({
#             "skills": [],
#             "technologies": [],
#             "certifications": [],
#             "notable_projects": [],
#             "years_experience": "",
#             "education_level": ""
#         })

#     time.sleep(1.5)  # delay to avoid hammering Groq service

# # Step 4: Convert results into a new DataFrame
# parsed_df = pd.DataFrame(parsed_outputs)

# # Step 5: Merge with original accepted dataset (first 15 rows for demo)
# enriched_df = pd.concat([df_accepted.head(15).reset_index(drop=True), parsed_df], axis=1)

# # Display the enriched dataframe
# enriched_df.head()

In [1]:
# # Save for future

# enriched_df.to_csv("enriched_df.csv", index=False)

In [ ]:
import pandas as pd
enriched_df = pd.read_csv("enriched_df.csv")
enriched_df.tail(5)

,Gender,Resume,Job Roles,Job Description,skills,technologies,certifications,years_experience,education_level
145,Female,"Proficient in Customer Interaction, Patience, ...",Customer Service Representative,A Customer Service Representative is the face ...,"['Customer Interaction', 'Patience', 'Product ...",[],['Certified Help Desk Professional (CHDP)'],mid-level,Bachelors degree
146,Female,"Proficient in Project Management, Digital Mark...",Marketing Manager,The Marketing Manager is responsible for craft...,"['Project Management', 'Digital Marketing', 'M...",['Google Ads'],['Google Ads Certification'],mid-level,Bachelors degree
147,Female,"Proficient in Machine Learning, Scientific Wri...",AI Researcher,An AI Researcher is at the forefront of advanc...,"['problem-solving', 'machine learning', 'model...",['machine learning frameworks'],['IBM AI Engineering Professional Certificate'],senior-level,Masters degree
148,Male,"Proficient in SQL, Business Intelligence, Proj...",Business Analyst,"As a Business Analyst, you will be responsible...","['project management', 'requirements gathering...","['sql', 'excel', 'business intelligence']",['Certified ScrumMaster (CSM)'],mid-level,Bachelors degree
149,Female,"Proficient in Innovation, Programming, Model B...",AI Researcher,An AI Researcher is at the forefront of advanc...,[],[],[],NaN,NaN


In [ ]:
import pandas as pd

df = pd.read_csv("enriched_df.csv")
print(df.columns.tolist())


['Gender', 'Resume', 'Job Roles', 'Job Description', 'skills', 'technologies', 'certifications', 'years_experience', 'education_level']


# match prompt

In [ ]:
match_prompt = """
You are a highly skilled HR Assistant and Resume Analyst with extensive experience in evaluating resumes against job descriptions.
Your goal is to extract structured information in the format below.
In the data provided to you (enriched_df), each applicant was **accepted** for the **Job Role** they applied for.

Use both the **Resume** and the corresponding **Job Role** and **Job Description** fields to guide your extraction.
Focus on how the applicant’s qualifications align with the role and Job description.

{{
  "experience_match": score out of 10,
  "skills_match": score out of 10,
  "project_relevance": score out of 10,
  "tech_match": score out of 10,
  "industry_relevance": score out of 10,
  "ats_score": score out of 10,
  "relevancy": score out of 10,
}}

Then calculate the Total Score (out of 70)

Also provide the bellow 2 fields
{{
   "match_keywords": [list of high match keywords from resume],
   "skill_gaps": [list of missing skills/keywords from JD]
}}


- Return output in JSON format
- Just give json in response. dont say here is json etc. Only json response.

"""

In [ ]:
# Match Prompt Commented

# import time, json
# parsed_outputs = []

# for i, row in enriched_df.head.iterrows():
#     try:
#         # Pass Resume + Job Role + Job Description
#         user_input = f"""
#         Resume: {row['Resume']}
#         Job Role: {row['Job Roles']}
#         Job Description: {row['Job Description']}
#         Skills: {row['skills']}
#         Technologies: {row['technologies']}
#         Certifications: {row['certifications']}
#         Years of Experience: {row['years_experience']}
#         Education Level: {row['education_level']}
#         """

#         response = client.chat.completions.create(
#             model="openai/gpt-oss-120b",
#             messages=[
#                 {"role": "system", "content": match_prompt},
#                 {"role": "user", "content": user_input},
#             ],
#             temperature=0.2
#         )

#         # Try parsing response to JSON
#         response_text = response.choices[0].message.content.strip()
#         print(f"Response for index {i}: {response_text}")
#         response_json = json.loads(response_text)
#         parsed_outputs.append(response_json)

#     except Exception as e:
#         print(f"Error processing index {i}: {e}")
#         parsed_outputs.append({
#             "experience_match": "",
#             "skills_match": "",
#             "project_relevance": "",
#             "tech_match": "",
#             "industry_relevance": "",
#             "ats_score": "",
#             "relevancy": "",
#             "Total Score": "",
#             "match_keywords": [],
#             "skill_gaps": []
#         })

#     time.sleep(1.5)  # delay to avoid hammering Groq service

Response for index 0: {
  "experience_match": 9,
  "skills_match": 7,
  "project_relevance": 5,
  "tech_match": 6,
  "industry_relevance": 9,
  "ats_score": 7,
  "relevancy": 8,
  "total_score": 51,
  "match_keywords": [
    "budgeting",
    "supply chain optimization",
    "risk management",
    "logistics",
    "project management",
    "adaptability",
    "SCPro Certification"
  ],
  "skill_gaps": [
    "analytical skills",
    "inventory management",
    "negotiation",
    "cost management"
  ]
}
Response for index 1: {
  "experience_match": 9,
  "skills_match": 8,
  "project_relevance": 5,
  "tech_match": 5,
  "industry_relevance": 9,
  "ats_score": 8,
  "relevancy": 8,
  "total_score": 52,
  "match_keywords": [
    "logistics",
    "negotiation",
    "procurement",
    "supply chain optimization",
    "budgeting",
    "analytical skills",
    "contract negotiation",
    "cost management",
    "Lean Six Sigma Green Belt Certification"
  ],
  "skill_gaps": [
    "inventory manageme

In [ ]:
# Step 4: Convert results into a new DataFrame
parsed2_df = pd.DataFrame(parsed_outputs)

# Step 5: Merge with original accepted dataset (first 15 rows for demo)
enriched2_df = pd.concat([enriched_df.reset_index(drop=True), parsed2_df], axis=1)

# Display the enriched dataframe
enriched2_df.head()

,Gender,Resume,Job Roles,Job Description,skills,technologies,certifications,years_experience,education_level,experience_match,skills_match,project_relevance,tech_match,industry_relevance,ats_score,relevancy,total_score,match_keywords,skill_gaps
0,Male,"Proficient in Budgeting, Supply Chain Optimiza...",Supply Chain Manager,A Supply Chain Manager oversees the entire sup...,"['analytical skills', 'inventory management', ...",[],['SCPro Certification'],senior-level,Masters degree,9,7,5,6,9,7,8,51,"[budgeting, supply chain optimization, risk ma...","[analytical skills, inventory management, nego..."
1,Male,"Proficient in Logistics, Negotiation, Procurem...",Supply Chain Manager,A Supply Chain Manager oversees the entire sup...,"['logistics', 'negotiation', 'procurement', 's...",[],['Lean Six Sigma Green Belt Certification'],senior-level,Masters degree,9,8,5,5,9,8,8,52,"[logistics, negotiation, procurement, supply c...",[inventory management]
2,Female,"Proficient in 3D Modeling, Project Management,...",Architect,An Architect designs and oversees the construc...,"['project management', 'delivering results', '...",['3D Modeling'],['Certified Interior Designer (CID)'],mid-level,PhD,8,8,7,9,8,9,8,57,"[3D Modeling, Project Management, Sustainable ...","[building codes, client interaction, detailed ..."
3,Male,"Proficient in Penetration Testing, Incident Re...",Cybersecurity Analyst,"As a Cybersecurity Analyst, you will safeguard...","['analytical skills', 'technical expertise', '...","['network security', 'penetration testing', 'i...",['Certified Information Security Manager (CISM)'],mid-level,PhD,8,7,6,9,9,8,8,55,"[Penetration Testing, Incident Response, Netwo...","[monitoring network traffic, vulnerability ass..."
4,Male,"Proficient in Software Development, Java, Pyth...",Software Engineer,"As a Software Engineer, you will leverage your...","['problem-solving', 'analytical skills', 'adap...","['Java', 'Python', 'C++', 'Kubernetes']",['Certified Kubernetes Administrator'],senior-level,Masters degree,9,8,6,9,9,9,9,59,"[Java, Python, C++, Kubernetes, problem-solvin...","[system design, independent work, complex codi..."


In [ ]:
# #Save it for future Analysis
# enriched2_df.to_csv("enriched2_df.csv", index=False)

# Training (Using Backend.py File)

In [47]:
import pandas as pd

In [ ]:
import os

# Create folder if it doesn't exist
if not os.path.exists("models"):
    os.makedirs("models")
    print("models/ folder created.")
else:
    print("models/ folder already exists.")


models/ folder created.


backend.py file Displayed for Relevance:

In [3]:
# ============================
# backend.py
# ============================
"""
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import hstack

# ----------------------------
# Utility functions
# ----------------------------
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

STOPWORDS = set([
    "a","an","the","and","or","in","on","of","with","to","for","is",
    "are","as","by","from","at","be","this","that","will","you","your"
])

def extract_keywords(text, top_n=20):
    words = clean_text(text).split()
    words = [w for w in words if w not in STOPWORDS and not w.isnumeric() and len(w) > 2]
    freq = pd.Series(words).value_counts()
    return list(freq.index[:top_n])

def map_experience_to_numeric(experience_str):
    mapping = {"entry-level":1,"mid-level":5,"senior-level":8,"expert":10}
    return mapping.get(str(experience_str).lower(),0)

def map_education_to_numeric(education_str):
    mapping = {"bachelors degree":1,"masters degree":2,"phd":3}
    return mapping.get(str(education_str).lower(),0)

# ----------------------------
# Load enriched_df
# ----------------------------
def load_enriched_df(file_path="enriched_df.csv"):
    df = pd.read_csv(file_path)
    df['years_experience_numeric'] = df['years_experience'].apply(map_experience_to_numeric)
    df['education_numeric'] = df['education_level'].apply(map_education_to_numeric)

    role_encoder = LabelEncoder()
    df['job_role_encoded'] = role_encoder.fit_transform(df['Job Roles'].fillna('Unknown'))

    return df, role_encoder

# ----------------------------
# Feature engineering
# ----------------------------
def prepare_features(df):
    combined_text = df['Resume'].fillna('') + " " + df['Job Description'].fillna('')
    if 'notable_projects' in df.columns:
        combined_text += " " + df['notable_projects'].fillna('').apply(lambda x: " ".join(eval(x)) if x.strip() != '' else '')
    df['combined_text'] = combined_text

    vectorizer = TfidfVectorizer(max_features=1000)
    X_text = vectorizer.fit_transform(df['combined_text'])

    # Only keep meaningful numeric features
    num_features = np.vstack([
        df['years_experience_numeric'],
        df['education_numeric']
    ]).T

    X = hstack([X_text, num_features])
    return X, vectorizer

# ----------------------------
# Train model
# ----------------------------
def train_numeric_model(df):
    X, vectorizer = prepare_features(df)
    target_cols = ['experience_match','skills_match','project_relevance',
                   'tech_match','industry_relevance','ats_score','relevancy']
    y = df[target_cols]

    model = RandomForestRegressor(n_estimators=150, random_state=42)
    model.fit(X, y)

    return model, vectorizer

# ----------------------------
# Match & Scope logic
# ----------------------------
def compute_match_scope(resume_text, jd_text):
    resume_keywords = set(extract_keywords(resume_text, top_n=50))
    jd_keywords = set(extract_keywords(jd_text, top_n=50))
    match = list(resume_keywords & jd_keywords)
    scope = list(jd_keywords - resume_keywords)
    return match, scope

# ----------------------------
# Inference
# ----------------------------
def predict_candidate_score(model, vectorizer, role_encoder,
                            resume_text, jd_text, job_role, projects_text="",
                            years_experience=None, education_level=None):

    # Encode job role
    try:
        job_role_encoded = role_encoder.transform([job_role])[0]
    except:
        job_role_encoded = 0

    # Use numeric features if provided
    years_experience_numeric = map_experience_to_numeric(years_experience) if years_experience else 0
    education_numeric = map_education_to_numeric(education_level) if education_level else 0

    # TF-IDF features
    combined_text = clean_text(resume_text) + " " + clean_text(jd_text) + " " + clean_text(projects_text)
    X_text = vectorizer.transform([combined_text])

    # Numeric features
    num_features = np.array([[years_experience_numeric, education_numeric]])
    X_input = hstack([X_text, num_features])

    numeric_cols = ['experience_match','skills_match','project_relevance',
                    'tech_match','industry_relevance','ats_score','relevancy']
    numeric_preds = model.predict(X_input)[0]

    # Keyword match
    match_keywords, skill_gaps = compute_match_scope(resume_text, jd_text)

    overall_percentage = numeric_preds.sum() / (len(numeric_cols)*10) * 100

    output = {col: float(val) for col,val in zip(numeric_cols, numeric_preds)}
    output.update({
        "overall_percentage": overall_percentage,
        "match_keywords": match_keywords,
        "skill_gaps": skill_gaps
    })

    return output
    """

<>:18: SyntaxWarning: invalid escape sequence '\s'
<>:18: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-1178451577.py:18: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub(r"[^a-z0-9\s]", " ", text)


'\nimport pandas as pd\nimport numpy as np\nimport re\nfrom sklearn.feature_extraction.text import TfidfVectorizer\nfrom sklearn.ensemble import RandomForestRegressor\nfrom sklearn.preprocessing import LabelEncoder\nfrom scipy.sparse import hstack\n\n# ----------------------------\n# Utility functions\n# ----------------------------\ndef clean_text(text):\n    text = str(text).lower()\n    text = re.sub(r"[^a-z0-9\\s]", " ", text)\n    text = re.sub(r"\\s+", " ", text).strip()\n    return text\n\nSTOPWORDS = set([\n    "a","an","the","and","or","in","on","of","with","to","for","is",\n    "are","as","by","from","at","be","this","that","will","you","your"\n])\n\ndef extract_keywords(text, top_n=20):\n    words = clean_text(text).split()\n    words = [w for w in words if w not in STOPWORDS and not w.isnumeric() and len(w) > 2]\n    freq = pd.Series(words).value_counts()\n    return list(freq.index[:top_n])\n\ndef map_experience_to_numeric(experience_str):\n    mapping = {"entry-level":1,"

In [51]:
# Import functions from backend.py
from backend import load_enriched_df, train_numeric_model

In [52]:
# Load enriched_df
df, role_encoder = load_enriched_df("enriched2_df.csv")

In [53]:
model, vectorizer = train_numeric_model(df)

Numeric model, vectorizer, and Job Role encoder saved.


In [55]:
os.listdir("models")

['job_role_encoder.pkl', 'numeric_model.pkl', 'tfidf_vectorizer.pkl']

In [ ]:
import shutil
from google.colab import files

# Zip the folder
shutil.make_archive("models", 'zip', "models")

# Download the zip file
files.download("models.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
#Attempt at pkl files at right format - Commented for future use

# import pickle

# # numeric_model, vectorizer, job_role_encoder = your trained objects

# with open("numeric_model.pkl", "wb") as f:
#     pickle.dump(numeric_model, f)

# with open("tfidf_vectorizer.pkl", "wb") as f:
#     pickle.dump(vectorizer, f)

# with open("job_role_encoder.pkl", "wb") as f:
#     pickle.dump(job_role_encoder, f)


--- End of Backend Code ---